Copyright (c) 2024 Agnese Re <agnesere43@gmail.com>\
GitHub account: https://github.com/AgneseRe

In [1]:
import pandas as pd
import numpy as np
from icecream import ic
from itertools import product
from dataclasses import dataclass

In [2]:
cities = pd.read_csv("cities/china.csv", sep = ",", header = None, names = ["city", "x", "y"])

### Distance matrix
Given the coordinates *(x, y)* of each city, calculate the Euclidean distance between each pair of cities.

In [3]:
dist_matrix = np.zeros((len(cities), len(cities)))
for a, b in product(cities.iterrows(), repeat = 2):
    dist_matrix[a[0], b[0]] = (
        ((a[1]['x']-b[1]['x'])**2 + (a[1]['y']-b[1]['y'])**2)**.5 if a[0] != b[0]
        else np.inf
    )

city_names = np.array([c['city'] for _, c in cities.iterrows()])
city_indexes = np.array([i for i, _ in cities.iterrows()])

In [4]:
ic(dist_matrix[0])
city = 2
ic(city_names[city])
closest_city = np.argmin(dist_matrix[0])
ic(city_names[closest_city])
None

ic| dist_matrix[0]: array([        inf, 46.91584167, 45.97178488, 38.89805779, 24.35579602,
                            1.85180993, 13.64950828, 22.07313299, 19.50017949, 32.00731428,
                           18.01137696, 11.97351243,  5.9679561 , 28.53708114, 15.76400013,
                           51.14451779,  4.14077287, 29.70085517,  3.6473631 , 24.47851507,
                           21.80337818, 13.2770215 , 22.55430413,  4.54816446, 17.61680164,
                           24.15812029, 24.4009044 , 12.36572953,  2.75363396, 25.84326798,
                           29.9643655 , 11.95766282, 28.31240403,  6.49224152,  7.23553039,
                           15.85070976,  5.2941666 , 28.32587686, 12.12785935, 44.89369441,
                           12.84817886, 12.81284512, 16.17494668, 21.20249985, 12.40711489,
                           27.66479978,  2.31969826, 22.49574404, 17.39104379, 39.66913158,
                           20.97144374, 24.09375157, 12.00333287, 22.27993941, 1

In [5]:
visited = np.full(len(cities), False)
city = 0
visited[city] = True
dist = dist_matrix.copy()
tsp = list()
while not np.all(visited):
    tsp.append(int(city))
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    print(f"{city} {city_names[city]:10s} -> {closest} {city_names[closest]:10s} ({dist_matrix[city, closest]})")
    visited[closest] = True
    city = closest

print(f"{closest} {city_names[closest]:10s} -> {tsp[0]} {city_names[tsp[0]]:10s} ({dist_matrix[closest, tsp[0]]})")
tsp.append(tsp[0])
ic(tsp)
None

ic| tsp: [0,
          186,
          505,
          677,
          508,
          285,
          264,
          289,
          93,
          46,
          157,
          513,
          328,
          384,
          419,
          212,
          252,
          114,
          197,
          359,
          636,
          550,
          229,
          414,
          173,
          396,
          397,
          198,
          290,
          101,
          451,
          23,
          507,
          133,
          544,
          233,
          395,
          240,
          191,
          399,
          644,
          539,
          516,
          174,
          28,
          573,
          412,
          450,
          232,
          358,
          681,
          554,
          16,
          532,
          434,
          435,
          514,
          699,
          5,
          356,
          469,
          457,
          207,
          696,
          570,
          481,
          343,
    

0 Acheng     -> 186 Harbin     (0.37443290453697164)
186 Harbin     -> 505 Shuangcheng (0.5448853090330131)
505 Shuangcheng -> 677 Yushu      (0.5905929224093394)
677 Yushu      -> 508 Shulan     (0.5663920903402501)
508 Shulan     -> 285 Jishu      (0.1920937271229772)
285 Jishu      -> 264 Jilin city (0.486632128911567)
264 Jilin city -> 289 Jiutai     (0.7644459358836244)
289 Jiutai     -> 93 Dehui      (0.4049691346263343)
93 Dehui      -> 46 Changchun  (0.7470609078247976)
46 Changchun  -> 157 Gongzhuling (0.6489992295835112)
157 Gongzhuling -> 513 Siping     (0.5882176467941108)
513 Siping     -> 328 Liaoyuan   (0.8443340571124661)
328 Liaoyuan   -> 384 Meihekou   (0.6545991139621294)
384 Meihekou   -> 419 Panshi     (0.5590169943749418)
419 Panshi     -> 212 Huadian    (0.6906518659932781)
212 Huadian    -> 252 Jiaohe     (0.9604686356149327)
252 Jiaohe     -> 114 Dunhua     (0.9546203433826403)
114 Dunhua     -> 197 Helong     (1.1244998888394813)
197 Helong     -> 359 Longjing

          616,
          538,
          540,
          36,
          126,
          81,
          105,
          711,
          142,
          650,
          87,
          418,
          33,
          139,
          338,
          283,
          234,
          605,
          59,
          34,
          340,
          66,
          61,
          724,
          531,
          346,
          183,
          530,
          143,
          535,
          309,
          549,
          637,
          509,
          31,
          218,
          388,
          123,
          716,
          146,
          27,
          459,
          194,
          41,
          44,
          220,
          318,
          38,
          112,
          196,
          504,
          443,
          645,
          718,
          708,
          40,
          305,
          617,
          124,
          349,
          193,
          324,
          620,
          429,
          520,
          217,
          213,
         

In [6]:
tot_cost = 0.0
for c1, c2 in zip(tsp, tsp[1:]):
    tot_cost += dist_matrix[c1, c2]

print(f"\nThe total cost for visiting all the cities is {tot_cost:.3f}")
None


The total cost for visiting all the cities is 589.353


In [7]:
@dataclass
class Individual:
    chromosome: np.ndarray
    fitness: float = None

def calculate_fitness(individual: Individual) -> float:
    tot_cost = 0.0
    for c1, c2 in zip(individual.chromosome, np.concatenate((individual.chromosome[1:], individual.chromosome[:1]))):
        tot_cost += dist_matrix[c1, c2]
    return -tot_cost

In [8]:
POPULATION_SIZE = 200
population = []
# Create a population of 200 individuals
for i in range(POPULATION_SIZE):
    new_individual = city_indexes.copy()
    np.random.shuffle(new_individual)
    if tuple(new_individual) not in [tuple(individual.chromosome) for individual in population]:
        population.append(Individual(new_individual))

for individual in population:
    individual.fitness = calculate_fitness(individual)

best_path_index = np.argmax([i.fitness for i in population])
print(f"Best path no. {best_path_index}: {population[best_path_index]}")

Best path no. 46: Individual(chromosome=array([605, 637, 498, 215, 165, 286, 113, 544, 101, 507,  81, 227, 397,
       536, 388, 533, 477,  50, 442, 703, 273,  30, 146, 386, 319, 607,
       600, 461, 641, 459,  90, 170, 136, 276, 719, 358, 520, 503, 530,
       694, 521, 185, 400, 663, 307, 368, 301,  27, 628, 462, 240, 325,
       219, 456, 718, 384, 708, 421, 608, 666, 495, 261, 547, 580, 383,
       622, 306, 211, 634, 426, 112, 370, 548, 501, 246,  75,  83, 660,
       557,   0, 532, 553, 377, 606, 184, 145, 230, 598, 584, 646, 135,
       601, 509, 449, 160, 475, 147, 680, 594, 297, 212, 283, 121, 609,
        49, 387, 695, 488, 346, 640, 206, 572,  22, 504, 154, 444,  33,
       576, 610, 359, 514,  72, 581, 436, 105, 562, 685, 337, 527, 167,
       661, 250, 713, 106, 707, 195, 103, 340, 570, 643, 303, 701, 385,
       597, 296, 508, 197, 178, 500, 221, 229, 126, 222, 565,  47,  41,
       339, 333, 485, 374, 437, 582, 148, 125, 149,  36, 429, 555, 450,
       568,  69,  96, 20

In [9]:
def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key = lambda c: c.fitness, reverse = True)
    return candidates[0]

def swap_mutation(individual):
    new_individual = Individual(individual.chromosome, individual.fitness)
    i1, i2 = np.random.choice(len(individual.chromosome), 2, replace = False)
    new_individual.chromosome[i1], new_individual.chromosome[i2] = new_individual.chromosome[i2], new_individual.chromosome[i1]
    new_individual.fitness = calculate_fitness(new_individual)
    return new_individual

def order_crossover(parent1: Individual, parent2: Individual):
    parent1, parent2 = Individual(parent1.chromosome, parent1.fitness), Individual(parent2.chromosome, parent2.fitness)
    size = len(parent1.chromosome)

    child = np.full(size, -1)
    start, end = sorted(np.random.choice(size, 2, replace=False))
    child[start:end+1] = parent1.chromosome[start:end+1]
    remaining_cities = [city for city in parent2.chromosome if city not in child]
    
    j = 0
    for i in range(size):
        if child[i] == -1:  # Se è un segnaposto (-1)
            child[i] = remaining_cities[j]
            j += 1
    
    return Individual(child)

In [10]:
MAX_GENERATIONS = 100
OFFSPRING_SIZE = 10

for g in range(MAX_GENERATIONS):
    offspring = []
    for _ in range(OFFSPRING_SIZE):
        if np.random.random() < .3:
            p = parent_selection(population)
            o = swap_mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = order_crossover(p1, p2)
        offspring.append(o)
    for i in offspring:
        i.fitness = calculate_fitness(i)
    population.extend(offspring)
    population.sort(key = lambda i: i.fitness, reverse = True)
    population = population[:POPULATION_SIZE]

best_path_index = np.argmax([i.fitness for i in population])
print(f"Best path no. {best_path_index}: {population[best_path_index]}")

Best path no. 0: Individual(chromosome=array([520, 251, 252, 509,  42, 623, 283,  35, 353, 595, 423, 574, 265,
       267, 116, 532, 234, 673, 145, 682, 429,  40, 368, 599, 530, 336,
       470, 561, 121, 296, 557, 163,  62, 591, 494,  24, 436,  25, 542,
       700, 668, 667,  51, 428, 710, 501, 433, 588, 607,   4, 694, 204,
       449, 383, 253, 146, 604, 645, 153, 304,  54, 435, 254, 430,  56,
       228, 110, 320, 241, 334, 554, 131, 620, 505, 152, 468, 138, 698,
       705, 200, 301, 662,  45, 504, 488, 289, 596,  88, 584, 676, 642,
       489,  10, 149,  97, 202, 630, 363, 316, 215, 132,  69, 500, 661,
       427, 384, 342, 329, 635, 326, 527, 541, 670, 114, 414, 365, 327,
       456, 570, 341, 484, 621, 563, 128, 421, 366, 616, 506, 424, 487,
        43,  57, 594, 654, 354, 461, 689, 724, 157,  74, 458, 224,  94,
        52, 418, 104, 715, 205, 239, 307, 507, 223, 603, 136, 248,  67,
        28, 323, 608, 141, 193, 165, 638, 209, 706, 373,  29,  23,  77,
       337, 249, 402,  39